In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
import time
import torch.optim as optim
import os
import sys

from torchvision import transforms
from torchvision.transforms import InterpolationMode
from torch.utils.data import DataLoader

In [ ]:
PROJECT_PATH = '/content/drive/MyDrive/Colab Notebooks/MLME2025_project'
CITYSCAPES_DIR = '/content/drive/MyDrive/Cityspaces/'
GTA5_DIR = '/content/drive/MyDrive/GTA5/'
BEST_MODEL_SAVE_PATH = '/content/drive/MyDrive/Colab Notebooks/MLME2025_project/models/BiSeNet/checkpoints_training_AUG/AUG2_best_model_BiSeNet.pth'
LAST_EPOCH_SAVE_PATH = '/content/drive/MyDrive/Colab Notebooks/MLME2025_project/models/BiSeNet/checkpoints_training_AUG/AUG2_last_epoch_BiSeNet.pth'


H_CITYSCAPES = 512
W_CITYSCAPES = 1024

H_GTA5 = 720
W_GTA5 = 1280

NUM_CLASSES = 19
BATCH_SIZE = 2
NUM_WORKERS = 2
LEARNING_RATE = 0.025
MOMENTUM = 0.9
WEIGHT_DECAY = 0.0001

In [ ]:
os.chdir(PROJECT_PATH)
sys.path.append(os.getcwd())

from datasets.cityscapes import CityScapes
from datasets.gta5 import GTA5
from utils.augmentation import RandomCropPair
from utils.utils import poly_lr_scheduler_with_backbone, fast_hist, per_class_iou, mean_iou
from models.BiSeNet.build_bisenet import BiSeNet

In [ ]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
# AUG2: random crop the image into fix size (526, 957) with prob 0.5 applied both on images and labels
#       (thanks to the class in utils.augmentation)
aug_transform = RandomCropPair(size=(526, 957), p=0.5)


# transformers data

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((H_GTA5, W_GTA5), interpolation=InterpolationMode.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((H_CITYSCAPES, W_CITYSCAPES), interpolation=InterpolationMode.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# transformers label

label_transforms = {
    'train': transforms.Compose([
        transforms.Resize((H_GTA5, W_GTA5), interpolation=InterpolationMode.NEAREST),
        transforms.Lambda(lambda x: torch.from_numpy(np.array(x)).long())
    ]),
    'val': transforms.Compose([
        transforms.Resize((H_CITYSCAPES, W_CITYSCAPES), interpolation=InterpolationMode.NEAREST),
        transforms.Lambda(lambda x: torch.from_numpy(np.array(x)).long())
    ])
}



# datsets
GTA_to_Cityscapes_datasets = {
    'train': GTA5(
        data_path=GTA5_DIR,
        transform=data_transforms['train'],
        label_transform=label_transforms['train'],
        aug_transform=aug_transform # <-- AUG2: RandomCropPair
    ),
    'val': CityScapes(
        data_path=CITYSCAPES_DIR,
        split='val',
        transform=data_transforms['val'],
        label_transform=label_transforms['val']
    )

}

# dataloaders
GTA_to_Cityscapes_dataloaders = {
    x: torch.utils.data.DataLoader(
        GTA_to_Cityscapes_datasets[x],
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        shuffle=True,
        pin_memory=True
    )
    for x in ['train', 'val']
}

# dataset sizes
dataset_sizes = {x: len(GTA_to_Cityscapes_datasets[x]) for x in ['train', 'val']}


In [ ]:
def train_model(model, data_loader, dataset_sizes, criterion, optimizer,
                last_epoch_save_path, best_model_save_path, num_epochs=1,
                init_lr=0.01, prev_num_epoch=0, prev_best_miou=0,
                total_number_epochs=50):

    since = time.time()

    best_miou = prev_best_miou
    best_per_class_iou = None

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        since_epoch = time.time()

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0

            hist = np.zeros((NUM_CLASSES, NUM_CLASSES))
            miou = 0

            for batch in data_loader[phase]:
                inputs = batch['x']
                labels = batch['y']

                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)

                    if phase == 'train':
                        loss = criterion(outputs[0], labels)
                        running_loss += loss.item()
                        loss.backward()

                        optimizer.step()

                        preds = torch.argmax(outputs[0], dim=1)
                        hist += fast_hist(
                                 preds.cpu().data.numpy().flatten(),
                                 labels.cpu().data.numpy().flatten(),
                                 NUM_CLASSES
                                )

                    else:
                        loss = criterion(outputs, labels)
                        running_loss += loss.item()

                        preds = torch.argmax(outputs, dim=1)
                        hist += fast_hist(
                                 preds.cpu().data.numpy().flatten(),
                                 labels.cpu().data.numpy().flatten(),
                                 NUM_CLASSES
                                )

            ious = per_class_iou(hist) * 100
            miou = mean_iou(ious)

            epoch_loss = running_loss / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {miou:.4f}')


        # update learning rate with poly_lr_scheduler
        next_lr = poly_lr_scheduler_with_backbone(optimizer, init_lr, prev_num_epoch, total_number_epochs)
        prev_num_epoch += 1

        # save the best model
        if miou > best_miou:
            best_miou = miou
            best_per_class_iou = ious
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, best_model_save_path)

        time_epoch = time.time() - since_epoch
        print(f'Epoch complete in {time_epoch // 60:.0f}m {time_epoch % 60:.0f}s')
        print(f'Next Learning Rate: {next_lr}')
        print()

        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, last_epoch_save_path)

    time_elapsed = time.time() - since
    print('-' * 20)
    print()
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val MIOU: {best_miou:4f}')
    print(f'Best val per class IOU: {best_per_class_iou}')
    print()
    print(f'Total Epochs completed: {prev_num_epoch}')

    return model, time_elapsed, best_miou, best_per_class_iou

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
context_path = 'resnet18'

model = BiSeNet(num_classes=NUM_CLASSES, context_path=context_path)
model = model.to(device)

optimizer = optim.SGD(
    params=[
    {'params': model.context_path.parameters(), 'lr': LEARNING_RATE * 0.1, 'initial_lr': LEARNING_RATE * 0.1},
    {'params': [p for module in model.mul_lr for p in module.parameters()], 'lr': LEARNING_RATE, 'initial_lr': LEARNING_RATE}
    ],
    momentum=MOMENTUM,
    weight_decay=WEIGHT_DECAY
)

criterion = nn.CrossEntropyLoss(ignore_index=255)

# load previous checkpoint
checkpoint = torch.load(LAST_EPOCH_SAVE_PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 183MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 220MB/s]


In [ ]:
model, time_elapsed, best_miou, best_per_class_iou = train_model(
    model=model,
    data_loader=GTA_to_Cityscapes_dataloaders,
    dataset_sizes=dataset_sizes,
    criterion=criterion,
    optimizer=optimizer,
    last_epoch_save_path=LAST_EPOCH_SAVE_PATH,
    best_model_save_path=BEST_MODEL_SAVE_PATH,
    num_epochs=17,
    init_lr=LEARNING_RATE,
    prev_num_epoch=3,
    prev_best_miou=14.0629,
    total_number_epochs=50
)

Epoch 1/17
----------
train Loss: 0.1918 Acc: 37.6760
val Loss: 0.7981 Acc: 18.4405
Epoch complete in 41m 34s
Next Learning Rate: [0.0023645857984465808, 0.023645857984465803]

Epoch 2/17
----------
train Loss: 0.1684 Acc: 41.0202
val Loss: 1.3342 Acc: 14.3607
Epoch complete in 24m 57s
Next Learning Rate: [0.002319257946408321, 0.023192579464083207]

Epoch 3/17
----------
train Loss: 0.1614 Acc: 43.4269
val Loss: 1.2162 Acc: 15.8276
Epoch complete in 24m 56s
Next Learning Rate: [0.002273831440207406, 0.022738314402074057]

Epoch 4/17
----------
train Loss: 0.1618 Acc: 42.7589
val Loss: 1.3567 Acc: 19.4938
Epoch complete in 25m 0s
Next Learning Rate: [0.0022283038652225005, 0.022283038652225]

Epoch 5/17
----------
train Loss: 0.1477 Acc: 46.7398
val Loss: 0.9365 Acc: 19.3419
Epoch complete in 25m 38s
Next Learning Rate: [0.0021826726914292156, 0.02182672691429215]

Epoch 6/17
----------
train Loss: 0.1346 Acc: 49.4362
val Loss: 1.0139 Acc: 16.8279
Epoch complete in 24m 55s
Next Learnin